## File to build files for each route

In [73]:
import numpy as np
import pandas as pd

In [74]:
# read in data
ridership = pd.read_csv("MBTA_Bus_Ridership_by_Time_Period%2C_Season%2C_Route_Line%2C_and_Stop.csv", low_memory=False)
stops = pd.read_csv("PATI_Bus_Stops.csv")
missing = pd.read_csv("missing_stops.csv")

In [75]:
# subset to fall 2022
ridership_subset = ridership[ridership['season'] == 'Fall 2022']
ridership_subset = ridership_subset[ridership_subset['direction_id'] == 0]  # there are 2 directions for each route (0 and 1)
ridership_subset = ridership_subset.reset_index(drop=True)

In [76]:
missing.columns

Index(['X', 'Y', 'Stop_ID', 'Stop_Name'], dtype='object')

In [77]:
ridership[ridership["stop_id"] == 64]

,mode,season,route_id,route_name,route_variant,stop_sequence,direction_id,day_type_id,day_type_name,time_period_id,time_period_name,stop_name,stop_id,average_ons,average_offs,average_load,num_trips,ObjectId
4,3,Fall 2016,1,1,1-_-0,1,0,day_type_01,weekday,time_period_04,MIDDAY_BASE,DUDLEY STATION,64,20.685000,0.000000,20.990000,20,5
30,3,Fall 2016,1,1,1-_-0,1,0,day_type_01,weekday,time_period_05,MIDDAY_SCHOOL,DUDLEY STATION,64,14.721429,0.000000,14.928571,14,31
52,3,Fall 2016,1,1,1-_-0,1,0,day_type_01,weekday,time_period_07,EVENING,DUDLEY STATION,64,5.027273,0.000000,5.109091,22,53
78,3,Fall 2016,1,1,1-_-0,1,0,day_type_01,weekday,time_period_08,LATE_EVENING,DUDLEY STATION,64,3.190000,0.000000,3.430000,10,79
113,3,Fall 2016,1,1,1-_-1,28,1,day_type_01,weekday,time_period_03,AM_PEAK,DUDLEY STATION,64,0.000000,10.907692,0.023077,13,114
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
857150,3,Fall 2022,8,8,8-9-1,24,1,day_type_01,weekday,time_period_08,LATE_EVENING,NUBIAN STATION,64,0.700000,0.600000,2.800000,1,857151
857224,3,Fall 2022,8,8,8-9-1,24,1,day_type_01,weekday,time_period_05,MIDDAY_SCHOOL,NUBIAN STATION,64,3.500000,5.650000,11.225000,4,857225
857253,3,Fall 2022,8,8,8-9-1,24,1,day_type_02,saturday,time_period_10,OFF_PEAK,NUBIAN STATION,64,2.321053,2.915789,8.326316,19,857254
857358,3,Fall 2022,8,8,8-9-1,24,1,day_type_01,weekday,time_period_06,PM_PEAK,NUBIAN STATION,64,1.880000,5.100000,9.920000,5,857359


In [78]:
# combine in missing stops data
stops_col_subset = stops[['X', 'Y', 'Stop_ID', 'Stop_Name']]

stops_with_missing = pd.concat([stops_col_subset, missing], axis=0)

In [79]:
stops_with_missing[stops_with_missing.Stop_ID == 64]

,X,Y,Stop_ID,Stop_Name
0,-71.084604,42.328989,64,Nubian Station


In [80]:
# combine the two datasets
combined = ridership_subset.merge(stops_with_missing, how="left", left_on="stop_id", right_on="Stop_ID")
combined = combined.drop_duplicates()

# subset out columns we need for optimization and plotting
cols = ["route_id","stop_id","stop_sequence","average_ons","average_offs","average_load","time_period_id","X","Y","Stop_ID","Stop_Name"]
# dropped: "stop_name", 
combined_subset = combined[cols]

key_routes = [1, 15, 22, 23, 28, 32, 39, 57, 66, 71, 73, 77, 111, 116, 117]

In [81]:
# build route file for each dataset
for route_id in set(combined_subset.route_id):
    route_subset = combined_subset[combined_subset["route_id"] == route_id]
    route_subset = route_subset.sort_values(by=["stop_sequence", "time_period_id"])
    route_subset = route_subset.drop(["stop_sequence"], axis=1)
    route_subset = route_subset.drop_duplicates()
    route_subset.to_csv("route_data/data_route" + str(route_id) + "_time_periods.csv", index=False)

    try: # type casting needed
        if int(route_id) in key_routes:
            route_subset.to_csv("key_routes/data_route" + str(route_id) + "_time_periods.csv", index=False)
    except Exception:
        pass

In [82]:
# build grouped route file for each dataset
for route_id in set(combined_subset.route_id):
    route_subset = combined_subset[combined_subset["route_id"] == route_id]

    agg_dict = {"route_id": "first",
                "stop_sequence": "first",
                "average_ons": "mean",
                "average_offs": "mean",
                "average_load": "mean",
                # "stop_name": "first",
                "X": "first",
                "Y": "first",
                "Stop_ID": "first",
                "Stop_Name": "first"}

    route_subset_grouped = route_subset.groupby("stop_id", as_index=False).agg(agg_dict)
    route_subset_grouped = route_subset_grouped.sort_values(by=["stop_sequence"])
    route_subset_grouped = route_subset_grouped.drop(["stop_sequence"], axis=1)
    route_subset_grouped = route_subset_grouped.drop_duplicates()
    route_subset_grouped.to_csv("route_data_grouped/data_route" + str(route_id) + "_time_periods.csv", index=False)

    try:
        if int(route_id) in key_routes:
            route_subset_grouped.to_csv("key_routes_grouped/data_route" + str(route_id) + ".csv", index=False)
    except Exception:
        pass